# sensitivity of (e,e'pp)/(e,e'p) ratio to selection cuts

In [1]:
from notebook_tools import *
from matplotlib.ticker import NullFormatter,MultipleLocator, FormatStrFormatter
from pylab import *
majorLocator   = MultipleLocator(0.01)
%matplotlib inline

Welcome to ROOTaaS 6.06/02

RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



/usr/local/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
PmissBins = [[0.3,0.45]  , [0.45,0.55] , [0.55,0.65] , [0.65,0.75] , [0.75,1.0]]
pmiss_analysis_bins , pmiss_analysis_bins_center , pmiss_multiples_bins , pmiss_bin_center = [] , [] , [] , []
for i in range(len(PmissBins)):
    pmin , pmax = PmissBins[i][0] , PmissBins[i][1]
    for j in range(4):
        pmiss_multiples_bins.append( [ pmin + float(j*(pmax-pmin))/4 , pmin + float((j+1)*(pmax-pmin))/4 ])
    for j in range(2):
        pmiss_analysis_bins.append( [ pmin + float(j*(pmax-pmin))/2 , pmin + float((j+1)*(pmax-pmin))/2 ])

for i in range( len(pmiss_multiples_bins) ):
    pmin , pmax = pmiss_multiples_bins[i][0] , pmiss_multiples_bins[i][1]
    pmiss_bin_center.append(0.5*(pmin+pmax))
print 'pmiss_bin_center:',pmiss_bin_center
for i in range( len(pmiss_analysis_bins) ):
    pmin , pmax = pmiss_analysis_bins[i][0] , pmiss_analysis_bins[i][1]
    pmiss_analysis_bins_center.append(0.5*(pmin+pmax))
print 'pmiss_analysis_bins_center:',pmiss_analysis_bins_center

pmiss_bin_center: [0.31875, 0.35625, 0.39375, 0.43125, 0.4625, 0.48750000000000004, 0.5125, 0.5375000000000001, 0.5625, 0.5875000000000001, 0.6125, 0.6375, 0.6625000000000001, 0.6875, 0.7124999999999999, 0.7375, 0.78125, 0.84375, 0.90625, 0.96875]
pmiss_analysis_bins_center: [0.3375, 0.4125, 0.475, 0.525, 0.5750000000000001, 0.625, 0.675, 0.725, 0.8125, 0.9375]


# (e,e'pp)/(e,e'p) ratio from DATA with no acceptance correction

In [79]:
def eepp_eep_in_pmiss_bins(eep=TPlots() , eepp=TPlots(),                 
                               cut='',
                               cut_name='nominal_cuts',                      
                               target = 'C12',
                               pmiss_bins=None,                    
                              ):
    
    pmiss_bin_center , eep_in_pmiss_bins , eepp_in_pmiss_bins = [] , [] , []
    for i in range( len(pmiss_bins) ):
        pmin , pmax = pmiss_bins[i][0] , pmiss_bins[i][1]
        pmiss_bin_center.append(0.5*(pmin+pmax))
        if 'nominal' in cut_name:
            selection_cut = ROOT.TCut("(%f<Pmiss3Mag && Pmiss3Mag<%f)"%(pmin , pmax))
        else:
            selection_cut = ROOT.TCut("(%f<Pmiss3Mag && Pmiss3Mag<%f) && (%s)"%(pmin , pmax , cut))
        
        eep_in_pmiss_bins.append( eep.GetEntries(selection_cut) )
        eepp_in_pmiss_bins.append( eepp.GetEntries(selection_cut) )
    
    return pmiss_bin_center , np.array(eep_in_pmiss_bins) , np.array(eepp_in_pmiss_bins)



In [80]:
def eepp_eep_in_pmiss_bin(eep=TPlots() , eepp=TPlots(),                                                
                          target = 'C12',                               
                          pmin = None,
                          pmax = None,                                                  
                         ):
    
        selection_cut = ROOT.TCut("(%f<Pmiss3Mag && Pmiss3Mag<%f)"%(pmin , pmax))
        Neep = eep.GetEntries(selection_cut) 
        Neepp = eepp.GetEntries(selection_cut) 
        return Neep , Neepp

In [81]:
def ratios_in_pmiss_bin( pmin , pmax ):

    ratio , ratio_err = [], [] 
    for target in ['C12','Al27','Fe56','Pb208']:
        eep = TPlots('/Users/erezcohen/Desktop/DataMining/AnaFiles/Ana_eep_in_ppSRCCut_DATA_%s.root'%target , 'anaTree')
        eepp = TPlots('/Users/erezcohen/Desktop/DataMining/AnaFiles/Ana_ppSRCCut_DATA_%s.root'%target , 'anaTree')
        Neep , Neepp = eepp_eep_in_pmiss_bin(eep=eep , eepp=eepp,
                                             target=target,
                                             pmin=pmin, pmax=pmax)
        ratio.append(100.*float(Neepp)/Neep)
        ratio_err.append(ratio[-1] * np.sqrt( 1./Neepp + 1./Neepp ))
        eep.Close()
        eepp.Close()
        
    df_ratios_in_pmiss_bin = pd.DataFrame({'$p_{miss}^{min}$':float(pmin),'$p_{miss}^{max}$':pmax,
                                          "$^{12}C$":ratio[0],"$\Delta ^{12}C$":ratio_err[0],
                                          "$^{27}Al$":ratio[1],"$\Delta ^{27}Al$":ratio_err[1],
                                          "$^{56}Fe$":ratio[2],"$\Delta ^{56}Fe$":ratio_err[2],
                                          "$^{208}Pb$":ratio[3],"$\Delta ^{208}Pb$":ratio_err[3],
                                         } , index=[0])

    return df_ratios_in_pmiss_bin

In [90]:
del df_ratio
columns=['$p_{miss}^{min}$','$p_{miss}^{max}$'
                                 ,"$^{12}C$","$\Delta ^{12}C$"
                                 ,"$^{27}Al$","$\Delta ^{27}Al$"
                                 ,"$^{56}Fe$","$\Delta ^{56}Fe$"
                                 ,"$^{208}Pb$","$\Delta ^{208}Pb$"]
df_ratio = pd.DataFrame(columns=columns)
for i in range(len(pmiss_analysis_bins)):
    pmin , pmax = pmiss_analysis_bins[i][0] , pmiss_analysis_bins[i][1]
    df_ratio_bin = ratios_in_pmiss_bin( pmin , pmax )
    df_ratio = df_ratio.append( df_ratio_bin )

# for target,name in zip(['C12','Al27','Fe56','Pb208'],
#                        ['$^{12}C$','$^{27}$Al','$^{56}$Fe','$^{208}$Pb']):

#     pmiss_bin_center , eep_in_pmiss_bins , eepp_in_pmiss_bins = [] , [] , []
#     for i in range( len(pmiss_bins) ):
#         pmin , pmax = pmiss_bins[i][0] , pmiss_bins[i][1]
#         pmiss_bin_center.append(0.5*(pmin+pmax))
#         if 'nominal' in cut_name:
#             selection_cut = ROOT.TCut("(%f<Pmiss3Mag && Pmiss3Mag<%f)"%(pmin , pmax))
#         else:
#             selection_cut = ROOT.TCut("(%f<Pmiss3Mag && Pmiss3Mag<%f) && (%s)"%(pmin , pmax , cut))
        
#         eep_in_pmiss_bins.append( eep.GetEntries(selection_cut) )
#         eepp_in_pmiss_bins.append( eepp.GetEntries(selection_cut) )



In [108]:
# print df_ratio[columns].to_string(index=False,float_format='%.3f')
print df_ratio[columns].to_latex(index=False,float_format='%.3f').replace("textasciicircum", '^').replace("textbackslash","\\")

\begin{tabular}{rrrrrrrrrr}
\toprule
 \$p\_\{miss\}\^\{min\}\$ &  \$p\_\{miss\}\^\{max\}\$ &  \$\^\{12\}C\$ &  \$\\Delta \^\{12\}C\$ &  \$\^\{27\}Al\$ &  \$\\Delta \^\{27\}Al\$ &  \$\^\{56\}Fe\$ &  \$\\Delta \^\{56\}Fe\$ &  \$\^\{208\}Pb\$ &  \$\\Delta \^\{208\}Pb\$ \\
\midrule
            0.300 &             0.375 &     0.548 &            0.200 &      0.584 &             0.369 &      0.805 &             0.248 &       0.447 &              0.365 \\
            0.375 &             0.450 &     2.685 &            0.507 &      1.619 &             0.661 &      1.829 &             0.399 &       1.818 &              0.742 \\
            0.450 &             0.500 &     4.473 &            0.803 &      6.000 &             1.633 &      2.937 &             0.612 &       3.789 &              1.263 \\
            0.500 &             0.550 &     6.090 &            0.957 &      5.828 &             1.648 &      4.947 &             0.813 &       3.361 &              1.188 \\
            0.550 &          

In [8]:
def eepp_eep_ratio(eep=TPlots() , eepp=TPlots(), 
                   cut='',
                   cut_name='nominal_cuts',                      
                   target = 'C12', target_name = '$^{12}$C',                   
                   pmiss_bins=None,                    
                   do_plot=True,
                   do_save_plots=True):
    
    bins, Neep, Neepp = eepp_eep_in_pmiss_bins(eep=eep , eepp=eepp,                 
                                               cut=cut,
                                               cut_name=cut_name,                      
                                               target=target,
                                               pmiss_bins=pmiss_bins)
    ratio , ratio_err =  [] , [] 
    for i in range(len(bins)):
        ratio.append(100*float(Neepp[i])/Neep[i])
        ratio_err.append(ratio[-1] * np.sqrt(1./Neepp[i] + 1./Neep[i]))
     
    if do_plot:
        fig,ax = plt.subplots(figsize=(15,10))
        plt.errorbar(pmiss_bin_center,ratio,xerr=np.zeros(len(pmiss_bin_center)),yerr=ratio_err,fmt='ro', color='black',markersize=20)
        set_axes(ax , x_label=r'$|\vec{p}_{miss}|$ [GeV/c]',y_label= r"%s ${\frac{(e,e'pp)}{(e,e'p)}}$"%target_name + ' [%]')
        ax.grid()
        ax.set_ylim(0,1.02*np.max(ratio)+np.max(ratio_err))
        ax.set_xlim(0.25,1.01)
        
        if do_save_plots:
            fig.savefig('/Users/erezcohen/Desktop/DataMining/AccCorr_ppSRCAnalysisNote/all_figures/eeppeep_sensitivity_studies/eeppeep_measured_%s_%s.pdf'%(cut_name,target))

    return bins , ratio , ratio_err